# Module 11 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Reinforcement Learning with Value Iteration

These are the same maps from Module 1 but the "physics" of the world have changed. In Module 1, the world was deterministic. When the agent moved "south", it went "south". When it moved "east", it went "east". Now, the agent only succeeds in going where it wants to go *sometimes*. There is a probability distribution over the possible states so that when the agent moves "south", there is a small probability that it will go "east", "north", or "west" instead and have to move from there.

There are a variety of ways to handle this problem. For example, if using A\* search, if the agent finds itself off the solution, you can simply calculate a new solution from where the agent ended up. Although this sounds like a really bad idea, it has actually been shown to work really well in video games that use formal planning algorithms (which we will cover later). When these algorithms were first designed, this was unthinkable. Thank you, Moore's Law!

Another approach is to use Reinforcement Learning which covers problems where there is some kind of general uncertainty in the actions. We're going to model that uncertainty a bit unrealistically here but it'll show you how the algorithm works.

As far as RL is concerned, there are a variety of options there: model-based and model-free, Value Iteration, Q-Learning and SARSA. You are going to use Value Iteration.

## The World Representation

As before, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol) is a node and there are edges to the north, south, east and west (except at the edges).

There are quite a few differences between A\* Search and Reinforcement Learning but one of the most salient is that A\* Search returns a plan of N steps that gets us from A to Z, for example, A->C->E->G.... Reinforcement Learning, on the other hand, returns  a *policy* that tells us the best thing to do in **every state.**

For example, the policy might say that the best thing to do in A is go to C. However, we might find ourselves in D instead. But the policy covers this possibility, it might say, D->E. Trying this action might land us in C and the policy will say, C->E, etc. At least with offline learning, everything will be learned in advance (in online learning, you can only learn by doing and so you may act according to a known but suboptimal policy).

Nevertheless, if you were asked for a "best case" plan from (0, 0) to (n-1, n-1), you could (and will) be able to read it off the policy because there is a best action for every state. You will be asked to provide this in your assignment.

We have the same costs as before. Note that we've negated them this time because RL requires negative costs and positive rewards:

In [25]:
costs = { '.': -1, '*': -3, '^': -5, '~': -7}
costs

{'.': -1, '*': -3, '^': -5, '~': -7}

and a list of offsets for `cardinal_moves`. You'll need to work this into your **actions**, A, parameter.

In [26]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]
cardinal_moves

[(0, -1), (1, 0), (0, 1), (-1, 0)]

For Value Iteration, we require knowledge of the *transition* function, as a probability distribution.

The transition function, T, for this problem is 0.70 for the desired direction, and 0.10 each for the other possible directions. That is, if the agent selects "north" then 70% of the time, it will go "north" but 10% of the time it will go "east", 10% of the time it will go "west", and 10% of the time it will go "south". If agent is at the edge of the map, it simply bounces back to the current state.

You need to implement `value_iteration()` with the following parameters:

+ world: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
+ costs: a `Dict` of costs by terrain (this is part of R)
+ goal: A `Tuple` of (x, y) stating the goal state.
+ reward: The reward for achieving the goal state.
+ actions: a `List` of possible actions, A, as offsets.
+ gamma: the discount rate

you will return a policy: 

`{(x1, y1): action1, (x2, y2): action2, ...}`

Remember...a policy is what to do in any state for all the states. Notice how this is different than A\* search which only returns actions to take from the start to the goal. This also explains why reinforcement learning doesn't take a `start` state.

You should also define a function `pretty_print_policy( cols, rows, policy)` that takes a policy and prints it out as a grid using "^" for up, "<" for left, "v" for down and ">" for right. Use "x" for any mountain or other impassable square. Note that it doesn't need the `world` because the policy has a move for every state. However, you do need to know how big the grid is so you can pull the values out of the `Dict` that is returned.

```
vvvvvvv
vvvvvvv
vvvvvvv
>>>>>>v
^^^>>>v
^^^>>>v
^^^>>>G
```

(Note that that policy is completely made up and only illustrative of the desired output). Please print it out exactly as requested: **NO EXTRA SPACES OR LINES**.

* If everything is otherwise the same, do you think that the path from (0,0) to the goal would be the same for both A\* Search and Q-Learning?
* What do you think if you have a map that looks like:

```
><>>^
>>>>v
>>>>v
>>>>v
>>>>G
```

has this converged? Is this a "correct" policy? What are the problems with this policy as it is?


In [27]:
def read_world(filename):
    result = []
    with open(filename) as f:
        for line in f.readlines():
            if len(line) > 0:
                result.append(list(line.strip()))
    return result

---

In [28]:
import copy

### is_passable
`is_passable` Checks if the position (x, y) is within the bounds of the world and not blocked. **Used by**: [value_iteration](#value_iteration)

- **x**: (int): x Coordinate.
- **y**: (int): y Coordinate.
- **world**: (list): The world represented as a grid.

**Returns** bool: True if the position is passable, False otherwise

In [29]:
def is_passable(x, y, world):
    """Check if the position (x, y) is within the bounds of the world and not blocked.
    Args:        
        x (int): The x-coordinate.
        y (int): The y-coordinate.
        world (list of list of str): The world represented as a grid.
    Returns:     bool: True if the position is passable, False otherwise."""
    rows = len(world)
    cols = len(world[0])
    return 0 <= x < cols and 0 <= y < rows and world[y][x] != 'x'

In [30]:
test_world = [
    ['.', '*', 'x'],
    ['^', '~', '.'],
    ['.', '.', '.']
]

assert is_passable(0, 0, test_world) == True 
assert is_passable(2, 0, test_world) == False
assert is_passable(-1, 0, test_world) == False
assert is_passable(0, -1, test_world) == False



### get_next_state
`get_next_state` Gets the next state based on the current position (x, y) and action. **Used by**: [value_iteration](#value_iteration)

- **x**: (int): x Coordinate.
- **y**: (int): y Coordinate.
- **action** (tuple): A tuple representing the action to take.
- **world**: (list): The world represented as a grid.

**Returns** tuple: The new position (nx, ny) after applying the action, or the current position if the action is not passable.

In [31]:
def get_next_state(x, y, action, world):
    nx, ny = x + action[0], y + action[1]
    if not is_passable(nx, ny, world):
        return x, y
    return nx, ny

In [32]:
assert get_next_state(0, 0, (1, 0), test_world) == (1, 0)  
assert get_next_state(1, 1, (1, 0), test_world) == (2, 1)  
assert get_next_state(1, 0, (1, 0), test_world) == (1, 0)  
assert get_next_state(0, 0, (-1, 0), test_world) == (0, 0)

### expected_value
`expected_value` Gets the next state based on the current position (x, y) and action. **Used by**: [value_iteration](#value_iteration)

- **x**: (int): x Coordinate.
- **y**: (int): y Coordinate.
- **action** (tuple): A tuple representing the action to take.
- **V** (dict): The value function.
- **world**: (list): The world represented as a grid.
- **costs** (dict): The movement costs.
- **rewards** (dict): The rewards for reaching certain states.
- **goal** (tuple): The goal state.
- **gamma** (float): The discount factor.
- **actions** (list): The list of possible actions.
- **main_prob** (float): The probability of taking the main action.
- **other_prob** (float): The probability of taking other actions.

**Returns** float: The expected value of the action.

In [33]:
def expected_value(x, y, action, V, world, costs, rewards, goal, gamma, actions, main_prob=0.7, other_prob=0.1):
    total = 0
    for i, a in enumerate(actions):
        prob = main_prob if a == action else other_prob
        nx, ny = get_next_state(x, y, a, world)
        next_state = (nx, ny)
        reward = rewards if next_state == goal else costs.get(world[ny][nx], 0)
        total += prob * (reward + gamma * V[next_state])
    return total

In [34]:
reward = 10
gamma = 0.9

V = {(x, y): 0 for y in range(len(test_world)) for x in range(len(test_world[0]))}
ev1 = expected_value(0, 0, (1, 0), V, test_world, costs, reward, (2, 2), gamma, cardinal_moves)
assert isinstance(ev1, float)

V_impass = {(x, y): 0 for y in range(len(test_world)) for x in range(len(test_world[0]))}
ev3 = expected_value(1, 0, (1, 0), V_impass, test_world, costs, reward, (2, 2), gamma, cardinal_moves)
assert ev3 < 0

### value_iteration
`value_iteration` Performs value iteration to find the optimal policy in a grid world.

- **world** (list of list of str): The world represented as a grid.
- **costs** (dict): A dictionary mapping terrain types to their costs.
- **goal** (tuple): The coordinates of the goal state.
- **rewards** (float): The reward for reaching the goal.
- **actions** (list of tuple): A list of possible actions represented as tuples (dx, dy).
- **gamma** (float): The discount factor for future rewards.

**Returns** dict: A dictionary representing the optimal policy, mapping states to actions.

In [35]:
def value_iteration(world, costs, goal, rewards, actions, gamma):
    rows = len(world)
    cols = len(world[0])
    V = {(x, y): 0 for y in range(rows) for x in range(cols)}

    threshold = 1e-4
    while True:
        delta = 0
        V_new = copy.deepcopy(V)
        for y in range(rows):
            for x in range(cols):
                state = (x, y)
                terrain = world[y][x]
                if state == goal:
                    V_new[state] = rewards
                    continue
                if terrain == 'x':
                    continue
                max_value = float('-inf')
                for a in actions:
                    val = expected_value(x, y, a, V, world, costs, rewards, goal, gamma, actions)
                    if val > max_value:
                        max_value = val
                delta = max(delta, abs(max_value - V[state]))
                V_new[state] = max_value
        V = V_new
        if delta < threshold:
            break

    # Extract policy
    policy = {}
    for y in range(rows):
        for x in range(cols):
            state = (x, y)
            terrain = world[y][x]
            if state == goal or terrain == 'x':
                continue
            best_action = None
            best_value = float('-inf')
            for a in actions:
                val = expected_value(x, y, a, V, world, costs, rewards, goal, gamma, actions)
                if val > best_value:
                    best_value = val
                    best_action = a
            policy[state] = best_action
    return policy

In [36]:
def pretty_print_policy( cols, rows, policy, goal):
    # Map actions to symbols
    action_symbols = { (0,-1): '^', (1,0): '>', (0,1): 'v', (-1,0): '<' }
    for y in range(rows):
        line = ""
        for x in range(cols):
            state = (x, y)
            if state == goal:
                line += "G"
            elif state not in policy or policy[state] is None:
                line += "x"
            else:
                action = policy[state]
                line += action_symbols.get(action, "?")
        print(line)

In [37]:
reward = 10

## Value Iteration

### Small World

In [38]:
small_world = read_world( "small.txt")

In [39]:
goal = (len(small_world[0])-1, len(small_world)-1)
gamma = 0.9

small_policy = value_iteration(small_world, costs, goal, reward, cardinal_moves, gamma)

In [40]:
cols = len(small_world[0])
rows = len(small_world)

# pretty_print_policy(cols, rows, test_policy, goal)
pretty_print_policy(cols, rows, small_policy, goal)

v>>>>v
v<^>>v
v<v>>v
vvvx>v
vvvvvv
v>>>vv
>>>>>G


### Large World

In [41]:
large_world = read_world( "large.txt")

In [42]:
goal = (len(large_world[0])-1, len(large_world)-1) # Lower Right Corner FILL ME IN
gamma = 0.9

large_policy = value_iteration(large_world, costs, goal, reward, cardinal_moves, gamma)

In [43]:
cols = len(large_world[0])
rows = len(large_world)

pretty_print_policy( cols, rows, large_policy, goal)

^<<<<<v<<<v>>>>>>>>>>^<<<<<
^<<<<<<<<vvv>>>>>^xxxxxxx^^
^^^^xx^<vvv<<^>^^xxxv<<xx^^
^^^^<xxxvvv<<^^^^>>v<<<xx>v
^^^^<xx>>vv<<<>>>>>v<<xxx>v
^^^^xx>>>v<<<<^^>>>^<<<x>>v
v<<xx>>>>v<<xxx>>>^^^<<>>>v
v<<<>>>>>^<<<<xxxv^^^^<>>>v
<<<<<>>>^^^<<<xxv<>>^^>>>>v
^<<<<>>^^^^xxxx^<<<^^^>>>>v
^^<<>>>^<^xxx>>^<<<<xxx>>>v
^^<<>vv^<xx>>>^^^^^<<xx>>>v
vvvv>vv<<xx>^^^^>>^^<x>>>>v
vvvv>vv<<<>>^<<<^^^^<>>>>>>
vvv<xv<<<<<^^^^<vv>^^<x>>>^
vv<xxx^<<<<^xxx>>vv^^xx>>>^
vvxx>>^^<<>>vvxxxv<xxxv>>>^
<<<xx^^^^>>>>>vvxxxx^<<<^^^
^<^xxx^^>>>>>>vvvv>>^^^^^^^
^^^<xxx>>>>>>>>vv<v^^^^^^^^
^<<^<vxx>>>>>^x<<<<<^^^^^^^
^<<<<vvxxx>^xx^^^^^<<<^^^vv
^^<<<<<vvxxxx^^^^^^^<<<vvvv
^^^^^<vvv>>>^<<^^xx^^<<vvvv
^x^^^vvvvv^xxx^^xx^xx^>vvvv
^xxxvvvvvv<vxxxx>vvvxxxvvvv
>>>^>>>v<<<<<<<>>>>>v>>>>>G


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.